In [29]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="E:\Workspace\Python\Workspace\ML_AI\singular-server-393911-b92f5a618cf9.json"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from google.cloud import bigquery
client = bigquery.Client()

query = """
SELECT visitNumber, date, date, totals, device FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` 
"""

#function to estimate the query output's size in bytes
def estimate_query_size(query):

    job_config = bigquery.job.QueryJobConfig(dry_run=True)
    job = client.query(query, job_config=job_config)
    return f"Estimated Query size will be: {job.total_bytes_processed} bytes"


def execute_query(query):

    query_job = client.query(query)
    results = query_job.result()

    if results.total_rows == 0:
        print("Query returns an empty table")
        return None

    print("Query returns a valid table")
    return results

print(estimate_query_size(query))
results = execute_query(query)

if results:
    df = results.to_dataframe()
    print(df)


Estimated Query size will be: 1184160 bytes
Query returns a valid table
      visitNumber      date    date_1  \
0               1  20170801  20170801   
1               2  20170801  20170801   
2               1  20170801  20170801   
3               1  20170801  20170801   
4               1  20170801  20170801   
...           ...       ...       ...   
2551            1  20170801  20170801   
2552            1  20170801  20170801   
2553            3  20170801  20170801   
2554           16  20170801  20170801   
2555            2  20170801  20170801   

                                                 totals  \
0     {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...   
1     {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...   
2     {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...   
3     {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...   
4     {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...   
...                                                 ...   
2551  {'visits': 1, 'h

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   visitNumber  2556 non-null   Int64 
 1   date         2556 non-null   object
 2   date_1       2556 non-null   object
 3   totals       2556 non-null   object
 4   device       2556 non-null   object
dtypes: Int64(1), object(4)
memory usage: 102.5+ KB


In [32]:
# print(df['totals'].apply(lambda x: x['visits']))
value = df.loc[0, 'totals'].get('visits')
print(value)

1


In [37]:
#split the dictionary and adding the columns to df
df = pd.concat([df.drop(['totals', 'device'], axis=1), df['totals'].apply(pd.Series), df['device'].apply(pd.Series)], axis=1)

#ddropping columns with "not available in demo dataset" values
cols_to_drop = [col for col in df.columns if df[col].eq("not available in demo dataset").any()]
df.drop(cols_to_drop, axis=1, inplace=True)

SyntaxError: invalid syntax (2842464640.py, line 2)

In [42]:
df

,visitNumber,date,date_1,visits,hits,pageviews,timeOnSite,bounces,transactions,transactionRevenue,...,screenviews,uniqueScreenviews,timeOnScreen,totalTransactionRevenue,sessionQualityDim,browser,operatingSystem,isMobile,javaEnabled,deviceCategory
0,1,20170801,20170801,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,Chrome,Windows,False,None,desktop
1,2,20170801,20170801,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,Chrome,Windows,False,None,desktop
2,1,20170801,20170801,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,Chrome,Windows,False,None,desktop
3,1,20170801,20170801,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,Firefox,Windows,False,None,desktop
4,1,20170801,20170801,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,Chrome,Windows,False,None,desktop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,1,20170801,20170801,1.0,84.0,60.0,2374.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,75.0,Chrome,Macintosh,False,None,desktop
2552,1,20170801,20170801,1.0,100.0,72.0,1792.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,62.0,Chrome,Macintosh,False,None,desktop
2553,3,20170801,20170801,1.0,140.0,93.0,2493.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,80.0,Chrome,Macintosh,False,None,desktop
2554,16,20170801,20170801,1.0,156.0,112.0,4444.0,NaN,1.0,400210000.0,...,NaN,NaN,NaN,406210000.0,74.0,Chrome,Windows,False,None,desktop


In [41]:
df.groupby('devicecategory').aggregate('visits' : sum)
                  


,visitNumber,pageviews,newVisits
deviceCategory,,,
desktop,1742,8329.0,1222.0
mobile,725,2332.0,580.0
tablet,89,278.0,70.0
